In [2]:
import requests

def get_company_data(org_number):
    url = f"https://data.brreg.no/regnskapsregisteret/regnskap/{org_number}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

org_number = "911382008"
company_data = get_company_data(org_number)

if company_data:
    print(company_data)
else:
    print("Failed to fetch company data.")
    
# save the data to a file

import json

with open('output/company_data.json', 'w') as outfile:
    json.dump(company_data, outfile, indent=4)


# convert the data to a dataframe 


[{'id': 3212571, 'journalnr': '2022419739', 'regnskapstype': 'SELSKAP', 'virksomhet': {'organisasjonsnummer': '911382008', 'organisasjonsform': 'ASA', 'morselskap': True}, 'regnskapsperiode': {'fraDato': '2021-01-01', 'tilDato': '2021-12-31'}, 'valuta': 'NOK', 'avviklingsregnskap': False, 'oppstillingsplan': 'store', 'revisjon': {'ikkeRevidertAarsregnskap': False, 'fravalgRevisjon': False}, 'regnkapsprinsipper': {'smaaForetak': False, 'regnskapsregler': 'regnskapslovenAlminneligRegler'}, 'egenkapitalGjeld': {'sumEgenkapitalGjeld': 27880000000.0, 'egenkapital': {'sumEgenkapital': 11283000000.0, 'opptjentEgenkapital': {'sumOpptjentEgenkapital': 5105000000.0}, 'innskuttEgenkapital': {'sumInnskuttEgenkaptial': 6178000000.0}}, 'gjeldOversikt': {'sumGjeld': 16597000000.0, 'kortsiktigGjeld': {'sumKortsiktigGjeld': 8787000000.0}, 'langsiktigGjeld': {'sumLangsiktigGjeld': 7810000000.0}}}, 'eiendeler': {'sumEiendeler': 27880000000.0, 'omloepsmidler': {'sumOmloepsmidler': 9095000000.0}, 'anleggsm

In [6]:
import json
import pandas as pd

def extract_numerical_values(data, prefix=''):
    numerical_data = []
    
    for key, value in data.items():
        if isinstance(value, dict):
            numerical_data.extend(extract_numerical_values(value, prefix + key + '.'))
        elif isinstance(value, (int, float)) and not isinstance(value, bool):
            
            value = int(value)
            numerical_data.append((key, value))
            
    return numerical_data

def json_to_dataframe(json_data):
    flattened_data = [item for record in json_data for item in extract_numerical_values(record)]
    
    df = pd.DataFrame(flattened_data, columns=['Variable', 'Value'])
    
    return df

df = json_to_dataframe(company_data)

df


,Variable,Value
0,id,3212571
1,sumEgenkapitalGjeld,27880000000
2,sumEgenkapital,11283000000
3,sumOpptjentEgenkapital,5105000000
4,sumInnskuttEgenkaptial,6178000000
5,sumGjeld,16597000000
6,sumKortsiktigGjeld,8787000000
7,sumLangsiktigGjeld,7810000000
8,sumEiendeler,27880000000
9,sumOmloepsmidler,9095000000
